In [30]:
import pandas as pd
import urllib.request
import json
import numpy as np

In [31]:
pokedex = pd.read_csv('pokedex.csv')
moves = pd.read_csv('pokemon-moves.csv')
url = 'https://raw.githubusercontent.com/Deskbot/Pokemon-Learnsets/master/output/gen3.json'
with urllib.request.urlopen(url) as response:
   page_content = response.read()
learnset = json.loads(page_content)

In [32]:
#clean pokedex to 151 pokemons in generation 1 (pokemons available in firered/ leafgreen)
pokedex = pokedex.drop(pokedex.columns[0], axis = 1)
pokedex_final = pokedex[pokedex['generation']==1].groupby('pokedex_number').first().reset_index()
pokedex_final['lowercase_name'] = pokedex_final['name'].apply(lambda x: x.lower().replace(" ", ""))

In [33]:
#restructure learnset to long format
learnset_df = pd.DataFrame.from_dict(learnset, orient = 'index').reset_index()
learnset_df['level'] = learnset_df['level'].apply(lambda x: list(x.values())) 
learnset_final = pd.melt(learnset_df, id_vars = 'index', var_name = 'learn_by', value_name = 'move').explode('move')

In [34]:
#join learnset to move statistic table
#moves:'willowisp', 'doubleedge', 'mudslap', 'selfdestruct', 'lockon','softboiled' do not have matches! 
moves['move'] = moves['Name'].apply(lambda x: x.lower().replace(" ", ""))
moves_final = pd.merge(learnset_final, moves, on = 'move').drop(['Index'], axis = 1)
moves_final.columns = map(str.lower, moves_final.columns)

#to find moves with no matches, use left join and below code
#moves_final['move'][moves_final['name'].isna() & moves_final['move'].notnull()].unique()

In [35]:
pokedex_final.head()

,pokedex_number,name,german_name,japanese_name,generation,status,species,type_number,type_1,type_2,...,against_flying,against_psychic,against_bug,against_rock,against_ghost,against_dragon,against_dark,against_steel,against_fairy,lowercase_name
0,1,Bulbasaur,Bisasam,フシギダネ (Fushigidane),1,Normal,Seed Pokémon,2,Grass,Poison,...,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5,bulbasaur
1,2,Ivysaur,Bisaknosp,フシギソウ (Fushigisou),1,Normal,Seed Pokémon,2,Grass,Poison,...,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5,ivysaur
2,3,Venusaur,Bisaflor,フシギバナ (Fushigibana),1,Normal,Seed Pokémon,2,Grass,Poison,...,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5,venusaur
3,4,Charmander,Glumanda,ヒトカゲ (Hitokage),1,Normal,Lizard Pokémon,1,Fire,None,...,1.0,1.0,0.5,2.0,1.0,1.0,1.0,0.5,0.5,charmander
4,5,Charmeleon,Glutexo,リザード (Lizardo),1,Normal,Flame Pokémon,1,Fire,None,...,1.0,1.0,0.5,2.0,1.0,1.0,1.0,0.5,0.5,charmeleon


In [36]:
pokedex_final.status

0         Normal
1         Normal
2         Normal
3         Normal
4         Normal
         ...    
146       Normal
147       Normal
148       Normal
149    Legendary
150     Mythical
Name: status, Length: 151, dtype: object

In [37]:
moves_final.head()

,index,learn_by,move,name,type,category,contest,pp,power,accuracy,generation
0,bulbasaur,level,tackle,Tackle,Normal,Physical,Tough,35,40,None,1
1,ivysaur,level,tackle,Tackle,Normal,Physical,Tough,35,40,None,1
2,squirtle,level,tackle,Tackle,Normal,Physical,Tough,35,40,None,1
3,caterpie,level,tackle,Tackle,Normal,Physical,Tough,35,40,None,1
4,pidgey,level,tackle,Tackle,Normal,Physical,Tough,35,40,None,1


In [38]:
moves_final['power'] = pd.to_numeric(moves_final["power"], downcast="float", errors = 'coerce')
#average power of moves by each pokemon
avg_power = pd.DataFrame(moves_final.groupby('index')['power'].mean()).reset_index()

In [39]:
avg_power.loc[avg_power.power.isna(), 'power'] = 0

In [40]:
#pokemon names that cannot be matched: nidoranf and nidoranm, farfetch'd, mr.mime (need to remove ' and .)
pokedex_final['lowercase_name'] = pokedex_final['lowercase_name'].str.replace('.', '').str.replace("'", '')
pokedex_final.loc[pokedex_final.pokedex_number == 29, 'lowercase_name'] = 'nidoranf'
pokedex_final.loc[pokedex_final.pokedex_number == 32, 'lowercase_name'] = 'nidoranm'
data = pd.merge(pokedex_final, avg_power, left_on = 'lowercase_name', right_on = 'index')

<ipython-input-40-0c99b8fd61a2>:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  pokedex_final['lowercase_name'] = pokedex_final['lowercase_name'].str.replace('.', '').str.replace("'", '')


In [41]:
#calculating damage constant for each pokemon in pokedex 
level = 60
ev = 100
iv = 15 
data['calculated_hp'] = (2*data.hp + iv + ev/4)*level/100 + level + 10
data['calculated_attack'] = (2*data.attack + iv + ev/4)*level/100 + 5 
data['calculated_defense'] = (2*data.defense + iv + ev/4)*level/100 + 5
data['calculated_speed'] = (2*data.speed + iv + ev/4)*level/100 + 5

In [42]:
#create 151 by 151 matrix for modifier for damage i inflicts on j (j's against_typeofpokemoni)
data = data.rename({'against_fight': 'against_fighting'}, axis = 1)

pokemon_types = ["against_" + x.lower() for x in data.type_1]
modifier_type1 = []
for pokemon_i in range(len(data)):
    row_i = [data.iloc[pokemon_j][pokemon_types[pokemon_i]] for pokemon_j in range(len(data))]
    modifier_type1.append(row_i)
modifier_type1 = pd.DataFrame(modifier_type1)

pokemon_types2 = ["against_" + x.lower() if isinstance(x, str) else 'NA' for x in data.type_2]
modifier_type2 = []
for pokemon_i in range(len(data)):
    row_i = [data.iloc[pokemon_j][pokemon_types2[pokemon_i]] if pokemon_types2[pokemon_i] != 'NA' else np.nan for pokemon_j in range(len(data))]
    modifier_type2.append(row_i)
modifier_type2 = pd.DataFrame(modifier_type2)

#make more effective "type"
modifier = pd.concat((modifier_type1, modifier_type2)).groupby(level = 0).max()

In [43]:
#create 151 by 151 matrix for damage i inflicts on j
d = pd.DataFrame(0, index=np.arange(len(data)), columns=np.arange(len(data)))
for i in range(len(data)):
    for j in range(len(data)):
        d.iloc[i,j] = ((2*level/5 * data.power[i] * data.calculated_attack[i]/data.calculated_defense[j])/50 + 2)*modifier.iloc[i,j]

In [44]:
#create 151 by 151 matrix for # turns i needs to defeat j
x = pd.DataFrame(0, index=np.arange(len(data)), columns=np.arange(len(data)))
for i in range(len(data)):
    for j in range(len(data)):
        if d.iloc[i,j] == 0:
            x.iloc[i,j] = 1000
        else: x.iloc[i,j] = data.calculated_hp[j] / d.iloc[i,j]

In [72]:
# type_1, type_2
data.columns
data[data.pokedex_number == 5].hp.iloc[0]

58.0

In [45]:
#create 151 by 151 matrix for # turns j needs to defeat i
y = pd.DataFrame(0, index=np.arange(len(data)), columns=np.arange(len(data)))
for i in range(len(data)):
    for j in range(len(data)):
        if d.iloc[j,i] == 0:
            y.iloc[i,j] = 1000
        else: y.iloc[i,j] = np.ceil(data.calculated_hp[i] / d.iloc[j,i])

In [77]:
x.transpose()

,0,1,2,3,4,5,6,7,8,9,...,141,142,143,144,145,146,147,148,149,150
0,4.109126,3.787488,2.694172,1.891656,1.609439,1.263289,8.050430,6.438573,4.825099,9.127845,...,1.076658,2.035719,1.251621,1.195129,1.038754,2.987188,2.426764,0.846886,0.973904,1.075447
1,5.433040,5.011899,3.575147,2.503213,2.132840,1.677098,10.646474,8.532435,6.407505,11.915729,...,1.430704,2.705959,1.661707,1.587166,1.380606,3.960996,3.222511,1.126705,1.294845,1.429104
2,7.532229,6.956348,4.981699,3.474427,2.966346,2.338325,14.764415,11.866862,8.937465,16.233059,...,1.997482,3.779545,2.317061,2.214027,1.928066,5.513529,4.494667,1.575681,1.809149,1.995266
3,3.605064,3.321692,2.359964,6.636028,5.642451,2.212748,1.765558,1.410793,1.056307,16.106273,...,0.942533,1.781887,2.192253,2.093043,1.818550,2.617484,2.125096,1.482013,1.704770,1.882942
4,5.066372,4.672288,3.329574,9.334336,7.949154,3.123322,2.481801,1.987540,1.491461,22.323498,...,1.331772,2.518574,3.094592,2.955471,2.570098,3.689907,3.000428,2.096707,2.410161,2.660560
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,3.769943,3.474026,2.469195,6.940369,5.902464,2.315310,7.385457,5.903216,4.421258,8.405658,...,1.972714,1.864817,1.146942,2.190167,1.903158,1.369168,1.111839,0.775592,1.784170,1.970492
147,5.589480,5.156812,3.679977,10.302376,8.779842,3.452758,10.953362,8.780948,6.596063,12.236936,...,2.945892,2.785972,1.710550,3.267772,2.842814,2.038353,1.658662,1.160166,2.666350,2.942600
148,8.878712,8.205406,5.889793,16.393278,14.012660,5.531114,17.406796,14.014394,10.572995,37.884746,...,2.364329,4.474806,1.370272,5.238624,4.565073,3.257254,2.658508,1.866909,4.284709,4.723436
149,9.293755,8.586579,6.157489,8.577374,7.328160,5.781648,9.109585,7.329070,5.525393,9.955453,...,4.941203,4.675450,5.729239,5.475248,4.769943,6.812359,5.557361,3.900042,8.952956,9.871472


0

In [47]:
#pokedex number of opponents
opponents = [18,65,112,103,130,6]

In [53]:
from gurobipy import *

no_pokemons = len(data)
no_opponents = len(opponents)
#preparing an optimization model
mod = Model("pokemon")

#declaring variables
c = mod.addVars(no_pokemons, no_opponents, name='c')

#setting the objective such that it helps us end the game as fast as possible
#but if want to make it safer, we should maximise the turn difference:
#mod.setObjective(sum(c[i,j]*t.iloc[i,j] for i in range(no_pokemons) for j in range(no_opponents)), GRB.MAXIMIZE)
mod.setObjective(sum(c[i,j]*x.iloc[i,opponents[j]-1] for i in range(no_pokemons) for j in range(no_opponents)), GRB.MINIMIZE)

#adding constraints
#comment next line out if want to remove constraint of only one of each pokemon in pokedex
mod.addConstrs(sum(c[i,j] for j in range(no_opponents)) <= 1 for i in range(no_pokemons))
mod.addConstrs(sum(c[i,j] for i in range(no_pokemons)) == 1 for j in range(no_opponents))
#chosen pokemons must be able to defeat opponent (no negative turn difference)
mod.addConstrs(c[i,j]*t.iloc[i,opponents[j]-1] >= 0 for i in range(no_pokemons) for j in range(no_opponents))  

#for pokemons who are slower and turn difference < 1, they would be first defeated by opponent 
mod.addConstrs(c[i,j] == 0 for i in range(no_pokemons) for j in range(no_opponents) if (data.calculated_speed[j] >= data.calculated_speed[i]) & (t.iloc[i,opponents[j]-1] < 1))  
#solving the optimization problem
mod.optimize()

#print optimal value
print('\nOptimal: %g\n' % mod.objVal)

#print optimal solutions
print('Optimal Assignment:')
for index, v in c.items():
    if v.getAttr("x") == 1:
        print("Pokemon {i} should battle pokemon {j}".format(i = data.name[index[0]], j = data.name[opponents[index[1]]-1]))

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 1475 rows, 906 columns and 3037 nonzeros
Model fingerprint: 0x9256fb8c
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [8e-01, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1469 rows and 896 columns
Presolve time: 0.00s
Presolved: 6 rows, 10 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.8344195e+00   1.000000e+00   0.000000e+00      0s
       1    6.8425759e+00   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.01 seconds
Optimal objective  6.842575863e+00

Optimal: 6.84258

Optimal Assignment:
Pokemon Golem should battle pokemon Pidgeot
76
Pokemon Muk should battle pokemon Alakazam
89
Pokemon Rhydon should battle pokemon Charizard
112
Pokemon Pinsir should battle pokemon Exeggutor
127
Poke

## TRY 2 can't work :(

In [ ]:
#calculated hp for pokemon i
def hp(i, level, iv, ev): 
    return((2*data.hp[i] + iv + ev/4)*level/100 + level + 10)
def attack(i, level, iv, ev):
    return((2*data.attack[i] + iv + ev/4)*level/100 + 5)
def defense(i, level, iv, ev):
    return((2*data.defense[i] + iv + ev/4)*level/100 + 5)
def speed(i, level, iv, ev):
    return((2*data.speed[i] + iv + ev/4)*level/100 + 5)

In [21]:
#create 151 by 151 matrix for modifier for damage i inflicts on j (j's against_typeofpokemoni)
pokemon_types = ["against_" + x.lower() for x in data.type_1]
modifier_type1 = []
for pokemon_i in range(len(data)):
    row_i = [data.iloc[pokemon_j][pokemon_types[pokemon_i]] for pokemon_j in range(len(data))]
    modifier_type1.append(row_i)
modifier_type1 = pd.DataFrame(modifier_type1)

pokemon_types2 = ["against_" + x.lower() if isinstance(x, str) else 'NA' for x in data.type_2]
modifier_type2 = []
for pokemon_i in range(len(data)):
    row_i = [data.iloc[pokemon_j][pokemon_types2[pokemon_i]] if pokemon_types2[pokemon_i] != 'NA' else np.nan for pokemon_j in range(len(data))]
    modifier_type2.append(row_i)
modifier_type2 = pd.DataFrame(modifier_type2)

modifier = pd.concat((modifier_type1, modifier_type2)).groupby(level = 0).max()

In [22]:
def damage(i,j,level,iv,ev):
    return(((2*level/5 * data.power[i] * attack(i,level,iv,ev)/defense(j,level,iv,ev))/50 + 2)*modifier.iloc[i,j])

In [23]:
#pokedex number of opponents
opponents = [18,65,112,103,130,6]

In [24]:
ev = 100
iv = 15

In [25]:
modifier 

,0,1,2,3,4,5,6,7,8,9,...,141,142,143,144,145,146,147,148,149,150
0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,2.0,2.0,2.0,0.5,0.5,0.5,0.5,0.5,0.5,2.0,...,0.5,1.0,2.0,1.0,0.5,0.5,0.5,0.5,1.0,1.0
4,2.0,2.0,2.0,0.5,0.5,0.5,0.5,0.5,0.5,2.0,...,0.5,1.0,2.0,1.0,0.5,0.5,0.5,0.5,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0
147,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0
148,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,...,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0
149,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,0.5,0.5


In [28]:
#speed not taken into account yet
from gurobipy import *
no_pokemons = len(data)
no_opponents = len(opponents)
#preparing an optimization model
mod = Model("pokemon")
mod.params.NonConvex = 2
#declaring variables
c = mod.addVars(no_pokemons, no_opponents, name='c')
t = mod.addVars(no_pokemons, no_pokemons, name = 'turn')

level = mod.addVar(1)
#ev = mod.addVar(1)
#iv = mod.addVar(1)
hp = mod.addVars(no_pokemons, name = 'hp')
attack = mod.addVars(no_pokemons, name = 'attack')
defense = mod.addVars(no_pokemons, name = 'defense')
speed = mod.addVars(no_pokemons, name = 'speed')
ad_ratio = mod.addVars(no_pokemons, no_pokemons, name = 'attack_defense_ratio')
damage = mod.addVars(no_pokemons, no_pokemons, name = 'damage_ij')
turns_x = mod.addVars(no_pokemons, no_pokemons, name = 'turns_you_defeat_opponent')
turns_y = mod.addVars(no_pokemons, no_pokemons, name = 'turns_opponent_defeat_you')

#setting the objective
mod.setObjective(sum(c[i,j]*t[i,opponents[j]-1] for i in range(no_pokemons) for j in range(no_opponents)), GRB.MAXIMIZE)

#adding constraints
mod.addConstrs(sum(c[i,j] for j in range(no_opponents)) <= 1 for i in range(no_pokemons))
mod.addConstrs(sum(c[i,j] for i in range(no_pokemons)) == 1 for j in range(no_opponents))

mod.addConstr(level == 60)
mod.addConstrs(hp[i] == (2*data.hp[i] + iv + ev/4)*level/100 + level + 10 for i in range(no_pokemons))
mod.addConstrs(attack[i] == (2*data.attack[i] + iv + ev/4)*level/100 + 5 for i in range(no_pokemons))
mod.addConstrs(defense[i] == (2*data.defense[i] + iv + ev/4)*level/100 + 5 for i in range(no_pokemons))
mod.addConstrs(speed[i] == (2*data.speed[i] + iv + ev/4)*level/100 + 5 for i in range(no_pokemons))
mod.addConstrs(attack[i] == ad_ratio[i,j] * defense[j] for i in range(no_pokemons) for j in range(no_pokemons))
mod.addConstrs(damage[i,j] == ((2*level/5 * data.power[i] * ad_ratio[i,j])/50 + 2)*modifier.iloc[i,j] for i in range(no_pokemons) for j in range(no_pokemons))
mod.addConstrs(hp[j] == turns_x[i,j]*damage[i,j] for i in range(no_pokemons) for j in range(no_pokemons))
mod.addConstrs(hp[i] == turns_y[i,j]*damage[j,i] for i in range(no_pokemons) for j in range(no_pokemons))
mod.addConstrs(t[i,j] == turns_y[i,j] - turns_x[i,j] for i in range(no_pokemons) for j in range(no_pokemons))

#solving the optimization problem
mod.optimize()

#print optimal value
print('\nOptimal: %g\n' % mod.objVal)

#print optimal solutions
print('Optimal Assignment:')
for index, v in c.items():
    if v.getAttr("x") == 1:
        print("Pokemon {i} should battle pokemon {j}".format(i = data.name[index[0]], j = data.name[opponents[index[1]]-1]))

Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 23563 rows, 115516 columns and 71424 nonzeros
Model fingerprint: 0x988b5329
Model has 906 quadratic objective terms
Model has 91204 quadratic constraints
Coefficient statistics:
  Matrix range     [5e-01, 6e+00]
  QMatrix range    [8e-02, 3e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+01]
  QRHS range       [5e-01, 8e+00]
Presolve removed 10827 rows and 22046 columns
Presolve time: 0.06s

Barrier solved model in 0 iterations and 0.06 seconds
Model is infeasible or unbounded


AttributeError: Unable to retrieve attribute 'objVal'